In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from runner import runner
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import GridSearchCV

In [2]:
split_year_month_bool = True # data_holidays, preprocessor, runner, weather upload edilecek.
train_path = "new_data/train.csv"
test_path = "new_data/test.csv"
weather_path = "new_data/weather.csv"
holidays_path = "new_data/holidays.csv"
train_df,test_df = runner(train_path,test_path,weather_path,holidays_path,split_year_month_bool)

In [3]:
def apply_label_encoder(train_data,test_data,columns):
    encoders = {}
    for col in columns:
        encoder = LabelEncoder()
        train_data[col] = encoder.fit_transform(train_data[col])
        test_data[col] = encoder.fit_transform(test_data[col])
        encoders[col] = encoder
    return train_data, test_data

In [4]:
def calculate_mae(true_values, predicted_values):
    """
    Bu fonksiyon, gerçek değerler ve tahmin edilen değerler arasındaki ortalama mutlak hatayı hesaplar.
    
    Args:
    true_values (array): Gerçek değerlerin bulunduğu dizi.
    predicted_values (array): Tahmin edilen değerlerin bulunduğu dizi.
    
    Returns:
    float: Hesaplanan ortalama mutlak hata.
    """
    # Gerçek ve tahmin edilen değerler arasındaki farkların mutlak değerlerini al
    absolute_errors = np.abs(true_values - predicted_values)
    
    # Mutlak hataların ortalamasını hesapla
    mae = np.mean(absolute_errors)
    
    return mae

In [5]:
test_df["bildirimsiz_sum"] = None

In [6]:
cities = train_df["il"].unique()
for city in cities:
    ilceler = train_df.loc[train_df["il"] == city]["ilce"].unique()
    for ilce in ilceler:
        print(f"---------------------{city},{ilce} için işlem yapılıyor.----------------------")
        ## test_df 
        ilce_test_data = test_df.loc[(test_df["il"] == city) & (test_df["ilce"] == ilce)]
        selected_indexes = ilce_test_data.index
        ## train_df
        ilce_train_data = train_df.loc[(train_df["il"] == city) & (train_df["ilce"] == ilce)]
        ## labelencoder
        ilce_train_encoded,ilce_test_encoded = apply_label_encoder(ilce_train_data,ilce_test_data,["il","ilce"])

        test_data = ilce_train_encoded[-29:]
        train_data = ilce_train_encoded[:-29]
        X_train = train_data.drop(["bildirimsiz_sum"], axis = 1)
        y_train = train_data["bildirimsiz_sum"].tolist()
        X_test = test_data.drop(["bildirimsiz_sum"], axis = 1)
        y_test = test_data["bildirimsiz_sum"].tolist()
        ## Model
        model = CatBoostRegressor(verbose=False)
        ## Parameters
        # param_grid = { # Bu parametreler daha iyi sonuç verdi.
        #     'depth': [3, 6, 9],  # Büyük derinlikler büyük veri setleri için daha uygundur, burada daha ılımlı derinlikler seçildi
        #     'learning_rate': [0.01, 0.05, 0.1],  # Yaygın olarak kullanılan öğrenme oranları
        #     'iterations': [50, 100, 150]  # 1000 satır için çok fazla iterasyon gereksiz olabilir
        # }
        param_grid = {
            'iterations': [100, 300, 500],
            'learning_rate': [0.01, 0.03, 0.05],
            'depth': [3, 4, 5],
        }
        # 'l2_leaf_reg': [3, 5, 7],
        # 'border_count': [50, 100, 150],
        # 'bagging_temperature': [0.5, 1, 1.5],
        # 'random_strength': [0.5, 1, 1.5]
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
        grid_search.fit(X_train, y_train)

        print("En iyi parametreler:", grid_search.best_params_)
        print("En iyi skor:", grid_search.best_score_)

        # Test veri seti üzerinde tahmin yap ve doğruluk skorunu hesapla
        best_model = grid_search.best_estimator_
        predictions = best_model.predict(X_test)
        mae_result = calculate_mae(y_test, predictions)
        print("Test doğruluk skoru:", mae_result)

        ## Gerçek Tahmin yap.
        real_predictions = best_model.predict(ilce_test_encoded)
        # test_df.loc[selected_indexes,"bildirimsiz_sum"] = np.round(real_predictions).astype(int)
        test_df.loc[selected_indexes,"bildirimsiz_sum"] = real_predictions

---------------------izmir,aliaga için işlem yapılıyor.----------------------
En iyi parametreler: {'depth': 3, 'iterations': 300, 'learning_rate': 0.03}
En iyi skor: -2.62011542757214
Test doğruluk skoru: 4.294030701829346
---------------------izmir,bayindir için işlem yapılıyor.----------------------
En iyi parametreler: {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
En iyi skor: -2.675386230574927
Test doğruluk skoru: 2.1388009303225344
---------------------izmir,bayrakli için işlem yapılıyor.----------------------
En iyi parametreler: {'depth': 3, 'iterations': 100, 'learning_rate': 0.05}
En iyi skor: -2.615433803833392
Test doğruluk skoru: 3.2114274558940394
---------------------izmir,bergama için işlem yapılıyor.----------------------
En iyi parametreler: {'depth': 4, 'iterations': 100, 'learning_rate': 0.03}
En iyi skor: -3.408960037079098
Test doğruluk skoru: 4.091178245051819
---------------------izmir,bornova için işlem yapılıyor.----------------------
En iyi parametr

In [7]:
test_df['tarih'] = pd.to_datetime(test_df[['year', 'month', 'day']])

In [8]:
def create_submission_df(test_df):
    test_df['tarih'] = test_df['tarih'].dt.strftime('%Y-%m-%d')
    submission_df = pd.DataFrame()
    submission_df['unique_id'] = test_df['tarih'] + '-' + test_df['il'] + '-' + test_df['ilce']
    submission_df["bildirimsiz_sum"] = test_df["bildirimsiz_sum"].tolist()
    return submission_df

In [9]:
submission_df = create_submission_df(test_df)

In [11]:
submission_df

,unique_id,bildirimsiz_sum
0,2024-02-01-izmir-aliaga,4
1,2024-02-02-izmir-aliaga,5
2,2024-02-03-izmir-aliaga,5
3,2024-02-04-izmir-aliaga,5
4,2024-02-05-izmir-aliaga,5
...,...,...
1358,2024-02-25-manisa-ahmetli,2
1359,2024-02-26-manisa-ahmetli,2
1360,2024-02-27-manisa-ahmetli,2
1361,2024-02-28-manisa-ahmetli,0


In [10]:
submission_df.to_csv("catboost_new_hyperparameters_rounded_with_new_data.csv", index = False)